# Introduction

This tutorial demonstrates basic steps involved in processing electron
backscatter diffraction (EBSD) patterns before crystal structure determination
(indexing) or other analysis.

For learning purposes, we recommend to use this notebook alongside our [user
guide](https://kikuchipy.org).

The data consists of 29 800 EBSD patterns from recrystallized polycrystalline
Nickel. It is available [via Zenodo](https://zenodo.org/record/3265037).

This functionality has been checked to run in kikuchipy-0.2.0 (May 2020). Bugs
are always possible, do not trust the code blindly, and if you experience any
issues, report them [in our issue
tracker](https://github.com/kikuchipy/kikuchipy-demos/issues).

Access all methods' documentation (docstrings) when the cursor is within the
function parantheses, like `function(CURSOR HERE)`, by holding `SHIFT` and
pressing `TAB`.

# Contents

1. [Loading and data inspection](#1)
2. [Background correction](#2)  
    1. [Static background correction](#2.1)  
    2. [Dynamic background correction](#2.2)
3. [Neighbour pattern averaging](#3)
4. [Further processing](#4)  
    1. [Intensity normalization](#4.1)  
    2. [Intensity rescaling/contrast stretching](#4.2)  
    3. [Filtering in the frequency domain (FFT filtering)](#4.3)  
    4. [Adaptive histogram equalization](#4.4)  

Import kikuchipy and other necessary libraries

In [ ]:
%matplotlib qt5
import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
import kikuchipy as kp

# 1. Loading and data inspection <a class="anchor" id="1"></a>

Load the EBSD data

In [ ]:
data = "/home/hakon/phd/data/ni/2020/1/nordif/Pattern.dat"
s = kp.load(data, lazy=False)  # lazy=True to not load into memory before calling s.compute()

Inspect the EBSD object `s`

In [ ]:
s

Inspect the data type

In [ ]:
s.data.dtype

Inspect the metadata associated with the EBSD object `s`

In [ ]:
s.metadata

Set important experimental parameters, like the pattern centre (in EMsoft's convention)

In [ ]:
s.set_experimental_parameters(
    xpc=38.86,
    ypc=132.65,
    zpc=16986.48,
    binning=8
)

See how this changed the metadata of the EBSD metadata node

In [ ]:
ebsd_node = kp.signals.util.metadata_nodes("ebsd")

In [ ]:
s.metadata.get_item(ebsd_node)

Set another metadata item and check that

In [ ]:
s.metadata.set_item("General.title", "Recrystallized Ni")

In [ ]:
s.metadata

In [ ]:
s

Inspect the signal (detector) and navigation (sample) axes

In [ ]:
s.axes_manager

Set the detector pixel size (and the detector origin to the detector centre)

In [ ]:
s.set_detector_calibration(delta=70)  # Microns

Inspect the axes manager again (note the signal scale)

In [ ]:
s.axes_manager

Plot the patterns

In [ ]:
s.plot()

If we want to only operate on or inspect parts of a data set, e.g. a square
from an upper left pattern (row, col) = (10, 20) to a bottom right pattern
(row, col) = (50, 70)

In [ ]:
#s = s.inav[10:50, 20:70]

# 2. Background correction <a class="anchor" id="2"></a>

## 2.A Static background correction <a class="anchor" id="2.1"></a>

Inspect the static background pattern

In [ ]:
static_bg = s.metadata.get_item(ebsd_node + ".static_background")

In [ ]:
plt.imshow(static_bg)

Remove the static background from each pattern by subtracting by it, but keeping relative intensities between the patterns

In [ ]:
s.remove_static_background(operation="subtract", relative=True)

Inspect the static background corrected patterns

In [ ]:
s.plot()

## 2.B Dynamic background correction <a class="anchor" id="2.2"></a>

Before we remove the large scale variations on the detector (dynamic background), we want to inspect it. It can be obtained by Gaussian blurring in real or frequency space 

In [ ]:
dynamic_bg = s.get_dynamic_background(
    filter_domain="frequency",  # or spatial
    std=8,
    truncate=4,
    dtype_out=np.float32,  # Takes up 4 times the RAM!
)

Inspect the dynamic background

In [ ]:
dynamic_bg.plot()

Remove the dynamic background from each pattern by subtracting by it (of course loosing relative intensities between the patterns)

In [ ]:
s.remove_dynamic_background(
    operation="subtract",
    filter_domain="frequency",
    std=8,
    truncate=4,
)

Inspect the dynamic background corrected patterns

In [ ]:
s.plot()

# 3. Neighbour pattern averaging <a class="anchor" id="3"></a>

Create an averaging kernel

In [ ]:
w_cross = kp.filters.Window(window="circular", shape=(3, 3))

Inspect the kernel and plot it

In [ ]:
w_cross.plot()

In [ ]:
w_cross

Create a Gaussian averaging kernel

In [ ]:
w_gauss = kp.filters.Window(window="gaussian", std=1)

Inspect it

In [ ]:
w_gauss

Average each pattern (in the window centre, `1.`) with the neighbouring patterns with the window coefficients in `w_gauss` (by performing spatial correlation)

In [ ]:
s.average_neighbour_patterns(window=w_gauss)

Inspect the averaged patterns

In [ ]:
s.plot()

# 4. Further processing <a class="anchor" id="4"></a>

## 4.A Intensity normalization <a class="anchor" id="4.1"></a>

Normalize the patterns intensities to a mean of zero and a standard deviation of one

In [ ]:
s.change_dtype(np.float32)  # Or passing dtype_out=np.float32 to normalize_intensity()

In [ ]:
s.normalize_intensity()

Inspect the normalized patterns

In [ ]:
s.plot()

## 4.B Intensity rescaling/contrast stretching <a class="anchor" id="4.2"></a>

Rescale pattern intensities to fill the entire data type range of the `uint16` data type

In [ ]:
s.rescale_intensity(dtype_out=np.uint16)

Inspect the rescaled patterns

In [ ]:
s.plot()

Perform contrast stretching on a copy of the data

In [ ]:
s2 = s.deepcopy()  # Double the memory!

In [ ]:
s2.rescale_intensity(percentiles=(0.5, 99.5))

Compare the patterns before and after contrast stretching

In [ ]:
hs.plot.plot_signals([s, s2])

## 4.C Filtering in the frequency domain <a class="anchor" id="4.3"></a>

Create a Gaussian lowpass transfer function of pattern shape to remove high frequency components (noise) in the patterns

In [ ]:
pattern_shape = s.axes_manager.signal_shape

In [ ]:
pattern_shape

In [ ]:
w_low = kp.filters.Window(
    window="lowpass",
    cutoff=22,
    cutoff_width=10,
    shape=pattern_shape
)

Inspect the filter

In [ ]:
plt.figure()
plt.imshow(w_low)

Create a Gaussian highpass transfer function of pattern shape to remove low frequency components (large scale variations,
a variant of dynamic background correction) in the pattern

In [ ]:
w_high = kp.filters.Window(
    window="highpass",
    cutoff=3,
    cutoff_width=2,
    shape=pattern_shape
)

Inspect the filter

In [ ]:
plt.figure()
plt.imshow(w_high)

Create a combined Gaussian high and low pass filter

In [ ]:
w = w_low * w_high

Inspect the combined filter

In [ ]:
plt.figure()
plt.imshow(w)

Apply the filter in the frequency domain on a copy of the data

In [ ]:
s3 = s.deepcopy()

s3.fft_filter(
    transfer_function=w,
    function_domain="frequency",  # It is already a transfer function
    shift=True,
)

Compare the patterns before and after FFT filtering

In [ ]:
hs.plot.plot_signals([s, s3])

## 4.D Adaptive histogram equalization <a class="anchor" id="4.4"></a>

Perform adaptive histogram equalization (like done in EMsoft's dictionary indexing program `EMEBSDDI`) on a copy of the data

In [ ]:
s4 = s.deepcopy()

In [ ]:
s4.adaptive_histogram_equalization()

Plot the two data sets before and after equalization side-by-side

In [ ]:
hs.plot.plot_signals([s, s4])